In [2]:
import pandas as pd
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import seaborn as sns   
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('data/data.csv')
df.head()

,participant,group,item_type,question_idx,item_idx,RT
0,0,both__allQ,word,0,0,493.752895
1,0,both__allQ,image,0,0,474.930928
2,0,both__allQ,word,0,1,481.147003
3,0,both__allQ,image,0,1,437.267612
4,0,both__allQ,word,0,2,566.318718


In [3]:
summary_stats = df.groupby(['question_idx', 'item_type'])['RT'].agg(['mean', 'std'])
print(summary_stats)

                              mean        std
question_idx item_type                       
0            image      486.325945  36.335796
             word       515.573713  36.835488
1            image      570.018701  39.852486
             word       600.157561  38.231441
2            image      321.999948  37.926035
             word       350.355810  37.132531
3            image      406.194478  39.420794
             word       436.163927  39.057984


In [8]:
# Fit a linear mixed-effects model
model = smf.mixedlm("RT ~ item_type * question_idx", df, groups=df["participant"])
result = model.fit()
print(result.summary())

                    Mixed Linear Model Regression Results
Model:                   MixedLM        Dependent Variable:        RT        
No. Observations:        1600           Method:                    REML      
No. Groups:              40             Scale:                     7023.8799 
Min. group size:         40             Log-Likelihood:            -9346.1610
Max. group size:         40             Converged:                 No        
Mean group size:         40.0                                                
-----------------------------------------------------------------------------
                                Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
-----------------------------------------------------------------------------
Intercept                      519.397    4.958 104.755 0.000 509.679 529.115
item_type[T.word]               29.371    7.012   4.189 0.000  15.628  43.114
question_idx                   -48.841    2.650 -18.429 0.000 -54.036 -43.647
item_t

/opt/homebrew/anaconda3/envs/wiscs-stats/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/anaconda3/envs/wiscs-stats/lib/python3.13/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/opt/homebrew/anaconda3/envs/wiscs-stats/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/anaconda3/envs/wiscs-stats/lib/python3.13/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
/opt/homebrew/anaconda3/envs/wiscs-stats/lib/python3.13/site-packages/statsmodels/base/model.py:607: 

In [15]:
from typing import Mapping, Tuple
from statsmodels.regression.mixed_linear_model import MixedLMResultsWrapper # type: ignore
def best_model(models: Mapping[any, MixedLMResultsWrapper], metric:str="aic", print_aic:bool=False) ->Tuple[any, MixedLMResultsWrapper]:
    """Return the best model based on metric"""
    assert metric in ["aic", "bic"], "Invalid metric"
    key = lambda item: getattr(item[1], metric)
    label, winner = min(models.items(), key=key)
    if print_aic:
        for model, result in models.items():
            print(f'{model}: {getattr(result, metric)}')
    return label, winner

In [17]:
model1 = smf.mixedlm("RT ~ item_type", df, groups=df["participant"])
result1 = model1.fit(method='lbfgs', reml=False)

model2 = smf.mixedlm("RT ~ question_idx", df, groups=df["participant"])
result2 = model2.fit(method='lbfgs', reml=False)

model3 = smf.mixedlm("RT ~ item_type * question_idx", df, groups=df["participant"])
result3 = model3.fit(method='lbfgs',reml=False)

In [21]:
models = {"RT ~ item_type":result1, 
          "RT ~ question_idx":result2, 
          "RT ~ item_type * question_idx":result3}
label, winner = best_model(models, print_aic=True)

print('-'*50)
print(f'\nwinner: {label}')
winner.summary() 

RT ~ item_type: 19282.879976598797
RT ~ question_idx: 18764.605525368082
RT ~ item_type * question_idx: 18719.912464546207
--------------------------------------------------

winner: RT ~ item_type * question_idx


<class 'statsmodels.iolib.summary2.Summary'>
"""
                    Mixed Linear Model Regression Results
=============================================================================
Model:                   MixedLM        Dependent Variable:        RT        
No. Observations:        1600           Method:                    ML        
No. Groups:              40             Scale:                     7006.3191 
Min. group size:         40             Log-Likelihood:            -9353.9562
Max. group size:         40             Converged:                 Yes       
Mean group size:         40.0                                                
-----------------------------------------------------------------------------
                                Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
-----------------------------------------------------------------------------
Intercept                      519.397    4.952 104.886 0.000 509.691 529.102
item_type[T.word]               29.371    7.003   4.194 0.000  15.645  43.097
question_idx                   -48.841    2.647 -18.452 0.000 -54.029 -43.653
item_type[T.word]:question_idx   0.038    3.743   0.010 0.992  -7.299   7.375
Group Var                        0.003                                       
=============================================================================

"""

np.float64(19282.914092289404)